This idea of this notebook is to create a first (simple) DL model using all the same features used in the multinomial logit model.

Much inspiration was derived from https://towardsdatascience.com/use-machine-learning-to-predict-horse-racing-4f1111fb6ced.

In [36]:
import math
from importlib import reload
import deeplearninglib
reload(deeplearninglib)
from deeplearninglib import *

import wandb

# device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cpu"
print(f"Using {device} device")

Using cpu device


In [37]:
# read in data

# select the same features used in the multinomial model
# X_columns = ["age", "sire_sr", "dam_sr", "trainer_sr", "daysLTO", "position1_1", "position1_2", "position1_3", "position1_4", "position2_1", "position2_2", "position2_3", "position2_4", "position3_1", "position3_2", "position3_3", "position3_4", "entire", "gelding", "blinkers", "visor", "cheekpieces", "tonguetie"]
# X_columns = ["sire_sr", "dam_sr", "trainer_sr", "position1_1", "position1_2", "position1_3", "position1_4", "position2_1", "position2_2", "position2_3", "position2_4", "position3_1", "position3_2", "position3_3", "position3_4", "entire", "gelding", "blinkers", "visor", "cheekpieces", "tonguetie"]
# X_columns = ["position1_1", "position1_2", "position1_3", "position1_4", "position2_1", "position2_2", "position2_3", "position2_4", "position3_1", "position3_2", "position3_3", "position3_4"]
X_columns = ["mkt_prob"]
y_columns = ["win"]

train_data_fn = "data\\runners_train.csv"
test_data_fn = "data\\runners_test.csv"

train_data = RacesDataset(train_data_fn, X_columns, y_columns)
test_data = RacesDataset(test_data_fn, X_columns, y_columns)

train_dataloader = DataLoader(train_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [38]:
pd.options.display.max_columns = 1000 # was 20
train_data.races.iloc[:, train_data.X_columns].head()

mkt_prob                                                    \
stall_number        1         2         3         4         5         6    
race_id                                                                    
11504         0.100000  0.038462  0.125000  0.058824  0.142857  0.053763   
11505         0.019608  0.029412  0.142857  0.038462  0.400000  0.058824   
11506         0.266667  0.038462  0.014925  0.058824  0.181818  0.038462   
11507         0.076923  0.058824  0.019608  0.222222  0.444444  0.029412   
11508         0.038462  0.038462  0.133333  0.047619  0.230947  0.200000   

                                                                               \
stall_number        7         8         9         10        11   12   13   14   
race_id                                                                         
11504         0.090909  0.125000  0.000000  0.000000  0.000000  0.0  0.0  0.0   
11505         0.053763  0.142857  0.029412  0.222222  0.153846  0.0  0.0  0.0   
11506         0.111111  0.029412  0.111111  0.307692  0.000000  0.0  0.0  0.0   
11507         0.125000  0.153846  0.066667  0.000000  0.000000  0.0  0.0  0.0   
11508         0.058824  0.181818  0.250000  0.000000  0.000000  0.0  0.0  0.0   

                        
stall_number   15   16  
race_id                 
11504         0.0  0.0  
11505         0.0  0.0  
11506         0.0  0.0  
11507         0.0  0.0  
11508         0.0  0.0

In [39]:
# build the neural network

output_layer_nodes = train_data.y.shape[1]
input_layer_nodes = train_data.X.shape[1]
bias = True

# torch.manual_seed(0)
model = net(input_layer_nodes, output_layer_nodes, bias=bias).to(device) # linear-relu-linear-softwax nn (1 hidden layer)
print(f"Model structure: {model}")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param} \n")

Model structure: net(
  (neural_network): Sequential(
    (0): Linear(in_features=16, out_features=16, bias=True)
    (1): Sigmoid()
  )
)
Layer: neural_network.0.weight | Size: torch.Size([16, 16]) | Values : Parameter containing:
tensor([[ 0.1277,  0.2205, -0.2487,  0.1354, -0.0726, -0.2448,  0.0555, -0.1361,
          0.0354, -0.1623, -0.1881, -0.0212, -0.0840,  0.1462,  0.0216,  0.1951],
        [ 0.0469, -0.0804,  0.1693,  0.0137,  0.0469, -0.0804,  0.0009, -0.0481,
         -0.0213, -0.1950, -0.0015, -0.2075, -0.0538,  0.1449, -0.1738, -0.1685],
        [-0.0610, -0.0685,  0.0423,  0.0415,  0.1268, -0.1722, -0.0176,  0.1398,
          0.2162, -0.0182, -0.1447,  0.0719,  0.1424, -0.1562, -0.1451,  0.1105],
        [-0.0175, -0.2361, -0.1441,  0.1014, -0.0848,  0.1726,  0.1976,  0.0364,
         -0.0198, -0.0794, -0.0126,  0.0455, -0.1910, -0.0597, -0.2080,  0.1534],
        [-0.1592,  0.2284, -0.0644, -0.1432,  0.1201,  0.0373,  0.1731,  0.1044,
         -0.2408,  0.1581, -0.0471,

In [40]:
# example to show how model is used from prediction

X = torch.rand(1, input_layer_nodes, device=device)
logits = model(X)
# pred_probab = nn.Softmax(dim=1)(logits)
# y_pred = pred_probab.argmax(1)
y_pred = logits.argmax(1)
print(f"Predicted class: {y_pred}")
print(X.shape)

Predicted class: tensor([6])
torch.Size([1, 16])


In [41]:
%env "WANDB_NOTEBOOK_NAME" "3b_Deep Learning.ipynb"

# optimizing model parameters

# hyperparameters
learning_rate = 10e-1
epochs = 50

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="horse-racing-project",
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": learning_rate,
    "architecture": list(model.modules()),
    "dataset": [train_data_fn, test_data_fn],
    "epochs": epochs,
    "bias": bias,
    "device": device,
    "X_columns": X_columns
    }
)

# initialize the loss function
loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer, device)
    (acc, loss) = test_loop(test_dataloader, model, loss_fn, device)
    wandb.log({"acc": acc, "loss": loss})
print("Done!")

# finish the wandb run, necessary in notebooks
wandb.finish()

env: "WANDB_NOTEBOOK_NAME"="3b_Deep Learning.ipynb"


Epoch 1
-------------------------------
loss: 2.756482  [   64/16620]
loss: 2.630939  [ 6464/16620]
loss: 2.609127  [12864/16620]
Test Error: 
 Accuracy: 23.3%, Avg loss: 2.583585 

Epoch 2
-------------------------------
loss: 2.646704  [   64/16620]
loss: 2.559203  [ 6464/16620]
loss: 2.569918  [12864/16620]
Test Error: 
 Accuracy: 23.5%, Avg loss: 2.558491 

Epoch 3
-------------------------------
loss: 2.637134  [   64/16620]
loss: 2.541376  [ 6464/16620]
loss: 2.557241  [12864/16620]
Test Error: 
 Accuracy: 23.2%, Avg loss: 2.547749 

Epoch 4
-------------------------------
loss: 2.632822  [   64/16620]
loss: 2.532133  [ 6464/16620]
loss: 2.550077  [12864/16620]
Test Error: 
 Accuracy: 22.9%, Avg loss: 2.540634 

Epoch 5
-------------------------------
loss: 2.629038  [   64/16620]
loss: 2.525591  [ 6464/16620]
loss: 2.544805  [12864/16620]
Test Error: 
 Accuracy: 22.4%, Avg loss: 2.534874 

Epoch 6
-------------------------------
loss: 2.625226  [   64/16620]
loss: 2.520219  [ 64

acc,▂▂▂▂▂▁▁▁▁▁▂▃▅▆▇▇████████████████████████
loss,█▇▇▆▆▆▆▅▅▅▅▅▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
acc,0.35414
loss,2.39767


In [42]:
for para_name, para_vals in model.named_parameters():
    np.savetxt(para_name + ".csv", para_vals.data.numpy(), fmt='%6.3f', delimiter=",")